In [35]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, isnan, when, count

In [2]:
spark = SparkSession.builder.appName("test").getOrCreate()

25/01/28 16:44:28 WARN Utils: Your hostname, Vasileioss-Laptop.local resolves to a loopback address: 127.0.0.1; using 192.168.0.67 instead (on interface en0)
25/01/28 16:44:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/28 16:44:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Load data

In [9]:
df=spark.read.csv('../data/cust_data.csv',inferSchema=True,header=True)

In [10]:
df.show()

+---+------+---+-------------------+---------+------------------------------+---------+--------+------------------+-------------------+-------+-----+
| id|Gender|Age|Has_Mobile_Contract|Area_Code|Currently_Holds_Second_Product|   Tenure|App_User|Num_website_visits|Acquisition_Channel|Revenue|Label|
+---+------+---+-------------------+---------+------------------------------+---------+--------+------------------+-------------------+-------+-----+
|  1|  Male| 44|                  1|       28|                             0|> 2 Years|     Yes|              NULL|                 26|  40.45|    1|
|  2|  Male| 76|                  1|        3|                             0| 1-2 Year|      No|              NULL|                 26|  33.54|    0|
|  3|  Male| 47|                  1|       28|                             0|> 2 Years|     Yes|              NULL|                 26|  38.29|    1|
|  4|  Male| 21|                  1|       11|                             1| < 1 Year|      No|    

In [12]:
print(f"Number of rows: {df.count()} and columns: {len(df.columns)}")

Number of rows: 381109 and columns: 12


In [13]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Has_Mobile_Contract: integer (nullable = true)
 |-- Area_Code: integer (nullable = true)
 |-- Currently_Holds_Second_Product: integer (nullable = true)
 |-- Tenure: string (nullable = true)
 |-- App_User: string (nullable = true)
 |-- Num_website_visits: string (nullable = true)
 |-- Acquisition_Channel: integer (nullable = true)
 |-- Revenue: double (nullable = true)
 |-- Label: integer (nullable = true)



In [15]:
df.describe().show()

25/01/28 16:58:11 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+------+-----------------+--------------------+------------------+------------------------------+---------+--------+------------------+-------------------+-----------------+-------------------+
|summary|                id|Gender|              Age| Has_Mobile_Contract|         Area_Code|Currently_Holds_Second_Product|   Tenure|App_User|Num_website_visits|Acquisition_Channel|          Revenue|              Label|
+-------+------------------+------+-----------------+--------------------+------------------+------------------------------+---------+--------+------------------+-------------------+-----------------+-------------------+
|  count|            381109|381109|           381109|              381101|            381108|                        381109|   381108|  381109|                 0|             381109|           381109|             381109|
|   mean|          190555.0|  NULL|38.82248385632457|   0.997869331227155| 26.38884778068159|            0.458210118

# Count missing values

In [38]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---+------+---+-------------------+---------+------------------------------+------+--------+------------------+-------------------+-------+-----+
| id|Gender|Age|Has_Mobile_Contract|Area_Code|Currently_Holds_Second_Product|Tenure|App_User|Num_website_visits|Acquisition_Channel|Revenue|Label|
+---+------+---+-------------------+---------+------------------------------+------+--------+------------------+-------------------+-------+-----+
|  0|     0|  0|                  8|        1|                             0|     1|       0|            381109|                  0|      0|    0|
+---+------+---+-------------------+---------+------------------------------+------+--------+------------------+-------------------+-------+-----+



# Continue with the exploration

In [21]:
df.groupBy('Gender').count().show()

+------+------+
|Gender| count|
+------+------+
|Female|175020|
|  Male|206089|
+------+------+



In [22]:
df.groupby('Has_Mobile_Contract').count().show()

+-------------------+------+
|Has_Mobile_Contract| count|
+-------------------+------+
|               NULL|     8|
|                  1|380289|
|                  0|   812|
+-------------------+------+

